In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd

import pims
import trackpy as tp
import os

import cv2

import matplotlib  as mpl 
import matplotlib.pyplot as plt 
import plotly.express as px

# Optionally, tweak styles.
mpl.rc('figure',  figsize=(15, 12)) #(10, 6)
mpl.rc('image', cmap='gray')

In [ ]:
my_img = pims.open('tiff_series_part_counter/*.tif')
my_img

In [ ]:
from stardist.models import StarDist2D, Config2D
from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label
from csbdeep.utils import normalize
import matplotlib.pyplot as plt

In [ ]:
model = StarDist2D.from_pretrained('2D_versatile_fluo')

In [ ]:
plt.figure(figsize=(9, 6))
plt.imshow(my_img[0], cmap='gray')
plt.axis('off')

In [ ]:
@pims.pipeline
def stardist_segm(img):
    img_labels, img_details = model.predict_instances(normalize(img, 1.0, 99.8), prob_thresh=0.75, nms_thresh=0.75)
    return img_labels

In [ ]:
label_image = stardist_segm(my_img)

In [ ]:
img_num = 30

plt.subplot(1, 2, 1)
plt.imshow(my_img[img_num], cmap='gray')
plt.axis('off')
plt.title('input image')

plt.subplot(1, 2, 2)
plt.imshow(render_label(label_image[img_num], img=my_img[img_num]))
# plt.imshow(he_labels)
plt.axis('off')
plt.title('prediction + input overlay')

In [ ]:
import skimage

In [ ]:
%%time
features = pd.DataFrame()

for num, img in enumerate(my_img):
    for region in skimage.measure.regionprops(label_image[num], intensity_image=img):
        # store fuatures
        dataset = pd.DataFrame({
            'y': [region.centroid[0]],
            'x': [region.centroid[1]],
            'frame': [num],
            'area': [region.area],
            'brightness': [region.intensity_mean],
        })
        
        features = pd.concat([features, dataset])

In [ ]:
features.head

In [ ]:
features.to_csv('data_out/S6_part_counter.csv', index=False)

In [ ]:
# read 'features' from file 
features = pd.read_csv('data_out/S6_part_counter.csv')

In [ ]:
# create copy of features to make modifications
features_modif = features.copy()

In [ ]:
tp.annotate(features_modif[features_modif.frame==(15)], my_img[15], plot_style={'markersize':2});

In [ ]:
# plot object areas to investigate the distribution. You can further filter out objects based on size or intensity. Not

# fig = px.histogram(features_modif['area'], nbins=20, template="simple_white")

# fig.show()

In [ ]:
# filter out by area (make the histogram looks more or less normal distrubution)

# features_modif = features_modif.loc[(features_modif['area'] > 40)]

In [ ]:
# plot new histogram

# fig = px.histogram(features_modif['area'], nbins=20, template="simple_white")

# fig.show()

In [ ]:
# plot the brightness (called as 'mass' in trackpy)

fig = px.histogram(features_modif['brightness'], template="simple_white")

fig.show()

In [ ]:
# filter out by intensity, then give it the name feature as it was originally

features_modif = features_modif.loc[(features_modif['brightness'] > 140)]

In [ ]:
# plot the brightness (called as 'mass' in trackpy)

fig = px.histogram(features_modif['brightness'], template="simple_white")

fig.show()

In [ ]:
tp.annotate(features_modif[features_modif.frame==(0)], my_img[0], plot_style={'markersize':7, 'marker':'x'})

In [ ]:
NOPdf = pd.DataFrame(columns=['slice', 'count'])

max_frame = features_modif.frame.iloc[-1]

for fr in range(max_frame + 1):
    mask = features_modif[features_modif.frame == fr]
    number = mask.x.count()
    
    NOPdf.loc[len(NOPdf.index)] = [fr, number]

NOPdf.head()

In [ ]:
# Save data to .dat file
np.savetxt('../scripts/input_data/slice_count_S6.dat', NOPdf, header="slice count", fmt='%d')